# Chapter 08: Tupperware

This chapter unveils the foundations of control flow, error handling, state, and effects, extending the power of pure functions.<br><br>


---
<br>

#### The Mighty Container:

In [1]:
class Container:
    def __init__(self, x):
        self.value = x
    
    @staticmethod
    def of(x):
        return Container(x)

In [2]:
print(Container.of(3).value)                                            # Container(3)

print(Container.of('hotdogs').value)                                    # Container("hotdogs")

print(Container.of(Container.of({ 'name': 'yoda' })).value.value)       # Container(Container({ name: 'yoda' }))

3
hotdogs
{'name': 'yoda'}


<u><b>Rules:</b></u>
- Container is an object with <u>one property</u>. <br>Lots of containers just hold <u>one thing</u>, though they aren't limited to one. <br>We've arbitrarily named its property <u>value</u>.

- The value <u>cannot be one specific type</u>.

- Once data goes <u>into</u> the Container it <u>stays there</u>. <br>We could get it out by using <u>.value</u>, but that would <u>defeat</u> the purpose.

---

<br>

#### My First Functor:

<b>Defintion</b><br>
In mathematics, specifically category theory, a functor is a mapping between categories.

We need a way to run functions on our container.

In [3]:
# (a -> b) -> Container a -> Container b
Container.map = lambda self, f: Container.of(f(self.value))

In [4]:
e1 = Container.of(2).map(lambda two: two + 2)
print(f"Expect: Container(4):\t\t\tOutput: {e1.value}")

e2 = Container.of('flamethrowers').map(lambda s: str.upper(s))
print(f"Expect: Container('FLAMETHROWERS'):\tOutput: {e2.value}")

e3 = Container.of('bombs').map(lambda s: f'{s} away').map(len)
print(f"Expect: Container(10):\t\t\tOutput: {e3.value}")

del e1, e2, e3

Expect: Container(4):			Output: 4
Expect: Container('FLAMETHROWERS'):	Output: FLAMETHROWERS
Expect: Container(10):			Output: 10


#### Schrödinger's Maybe :


In [5]:
from utils import curry, compose, trace
from functools import partial
import re

In [6]:
class Maybe:
    @staticmethod
    def of(x):
        return Maybe(x)

    def isNothing(self):
        return not self.value

    def __init__(self, x):
        self.value = x
  

    map = curry(lambda self, fn: self if self.isNothing() else Maybe.of(fn(self.value)))
  
    # The equivalent of inspect() in python is __str__ method
    def __str__(self):
        return 'Nothing' if self.isNothing() else f'Just({self.value})'

In [7]:
# Let's redefine our map function as we'll be using map from the class
map = curry(lambda f, xs: [f(s) for s in xs] if isinstance(xs, list) else xs.map(f))

match = curry(lambda reg, s:  bool(re.findall(reg, s)))
prop = curry(lambda p, obj: obj[p] if p in obj else None)
add = curry(lambda n, x: n+x)

In [8]:
e1 = Maybe.of('Malkovich Malkovich').map(match(r'a'))
print(f"Expect: Just(True)\tOutput: {e1}")

e2 = Maybe.of(None).map(match(r'a'))
print(f"Expect: Nothing\t\tOutput: {e2}")

e3 = Maybe.of({ 'name': 'Boris' }).map(prop('age')).map(add(10))
print(f"Expect: Nothing\t\tOutput: {e3}")

e4 = Maybe.of({ 'name': 'Dinah', 'age': 14 }).map(prop('age')).map(add(10))
print(f"Expect: Just(24)\tOutput: {e4}")

del e1, e2, e3, e4

Expect: Just(True)	Output: Just(True)
Expect: Nothing		Output: Nothing
Expect: Nothing		Output: Nothing
Expect: Just(24)	Output: Just(24)


<b>Note for map:</b><br>
```map :: Functor f => (a -> b) -> f a -> f b```

---

#### Use Cases


In [9]:
# safeHead :: [a] -> Maybe(a)
safeHead = lambda xs : Maybe.of(xs[0] if xs else None)

# streetName :: Object -> Maybe String
streetName = compose(map(prop('street')), safeHead, prop('addresses'))

In [10]:
e1 = streetName({ 'addresses': [] })
print(f"Expect: Nothing\t\t\tOutput: {e1}")

e2 = streetName({ 'addresses': [{ 'street': 'Shady Ln.', 'number': 4201 }] })
print(f"Expect: Just('Shady Ln.')\tOutput: {e2}")

del e1, e2

Expect: Nothing			Output: Nothing
Expect: Just('Shady Ln.')	Output: Just(Shady Ln.)


```null``` values are no longer affecting our code

In [11]:
# withdraw :: Number -> Account -> Maybe(Account)
withdraw = curry(lambda amount, account: 
                        Maybe.of(
                            { 'balance': account.get('balance') - amount } 
                            if account.get('balance') >= amount 
                            else None))

# This function is hypothetical, not implemented here... nor anywhere else.
# updateLedger :: Account -> Account 
updateLedger = lambda account: account

# remainingBalance :: Account -> String
remainingBalance = lambda account: f'Your balance is ${account.get('balance')}'

# finishTransaction :: Account -> String
finishTransaction = compose(remainingBalance, updateLedger)

# getTwenty :: Account -> Maybe(String)
getTwenty = compose(Maybe.map(fn=finishTransaction), withdraw(20))

In [12]:
e1 = getTwenty({ 'balance': 200.00 }) 
print(f"Expect: Just('Your balance is $180')\tOutput: {e1}")

e2 = getTwenty({ 'balance': 10.00 })
print(f"Expect: Nothing\t\t\t\tOutput: {e2}")

del e1, e2

Expect: Just('Your balance is $180')	Output: Just(Your balance is $180.0)
Expect: Nothing				Output: Nothing


Unlike before, where null values were undesirable, here they are used intentionally to cancel the <br>
transaction when funds are insufficient, preventing unintended updates to the ledger or balance.<br>

---

#### Releasing the Value:

Don't try to extract a value from a ```Maybe``` instance as if it will always exist.<br>
We must acknowledge that it may not - <u>Like Schrödinger’s cat !</u><br>

But we can use a helper we call ```maybe```:


In [13]:
# maybe :: b -> (a -> b) -> Maybe a -> b
maybe = curry(lambda v, f, m: v if m.isNothing() else f(m.value))

# getTwenty :: Account -> String
getTwenty = compose(maybe('You\'re broke!', finishTransaction), withdraw(20))

In [14]:
e1 = getTwenty({ 'balance': 200.00 })
print(f"Expect: 'Your balance is $180.00'\t\tOutput: {e1}")

e2 = getTwenty({ 'balance': 10.00 })
print(f"Expect: 'You\'re broke!'\t\t\t\tOutput: {e2}")

del e1, e2

Expect: 'Your balance is $180.00'		Output: Your balance is $180.0
Expect: 'You're broke!'				Output: You're broke!


##### <b>Summary:</b>
```Maybe``` (or ```Option```) – <b>Handles optional values</b>: 
- Represents a value that might be present (```Just x```) or missing (```Nothing```).
- Prevents ```None``` or ```null``` errors.
<br>
<br>
---

#### Pure Error Handling:

try/except is not very pure.<br>
Instead of raising Errors in an aggressive manner for an invalid input.<br>
We can gently communicate the issue.

In [15]:
class Either:
    @staticmethod
    def of(x):
        return Right(x)

    def __init__(self, x):
        self.value = x
    
class Left(Either):
    map = curry(lambda self, f: self)

    def __str__(self):
        return f'Left({self.value})'

class Right(Either):
    map = curry(lambda self, f: Either.of(f(self.value)))

    def __str__(self):
        return f'Right({self.value})'

left = lambda x: Left(x)

In [16]:
e1 = Either.of('rain').map(lambda str: f'b{str}') 
print(f"Expect: Right('brain')\t\t\t\tOutput: {e1}")
 
e2 = left('rain').map(lambda str: f"It's gonna {str}, better bring your umbrella!")
print(f"Expect: Left('rain')\t\t\t\tOutput: {e2}")

e3 = Either.of({ 'host': 'localhost', 'port': 80 }).map(prop('host'))
print(f"Expect: Right('localhost')\t\t\tOutput: {e3}")

e4 = left('rolls eyes...').map(prop('host'));
print(f"Expect: Left('rolls eyes...')\t\t\tOutput: {e4}")

del e1, e2, e3, e4

Expect: Right('brain')				Output: Right(brain)
Expect: Left('rain')				Output: Left(rain)
Expect: Right('localhost')			Output: Right(localhost)
Expect: Left('rolls eyes...')			Output: Left(rolls eyes...)


Let's customize our errors.

In [17]:
import datetime as dt
from dateutil.relativedelta import relativedelta

moment = dt.datetime.now

In [18]:
def getAge(now, user):
    try:
        birthDate = dt.datetime.strptime(prop('birthDate')(user), '%Y-%m-%d')
    except:
        birthDate = None
    
    return Either.of(relativedelta(now, birthDate).years) if birthDate else left('Birth date could not be parsed')

# getAge :: Date -> User -> Either(String, Number)
getAge = curry(getAge)

The try/except clause was used to replace birthDate with an alternative value.

In [19]:
e1 = getAge(moment(), { 'birthDate': '2005-12-12' })
print(f"Expect: Right(19)\t\t\t\tOutput: {e1}")

e2 = getAge(moment(), { 'birthDate': 'July 4, 2001' })
print(f"Expect: Left('Birth date could not be parsed')\tOutput: {e2}")

del e1, e2

Expect: Right(19)				Output: Right(19)
Expect: Left('Birth date could not be parsed')	Output: Left(Birth date could not be parsed)


In [20]:
# We can define our concat equivalent to a curried version using str.__add__
concat = curry(lambda s,t: s+t)

# fortune :: Number -> String
fortune = compose(concat('If you survive, you will be '), str, add(1))

# zoltar :: User -> Either(String, _)
zoltar = compose(map(print), map(fortune), getAge(moment()))

In [21]:
e1 = zoltar({ 'birthDate': '2005-12-12' })
print(f"{e1}\nExpect:\tIf you survive, you will be 10'\n\tRight(None)\n")


e2 = zoltar({ 'birthDate': 'balloons!' })
print(f"Expect: Left('Birth date could not be parsed')\nOutput: {e2}")

del e1, e2

If you survive, you will be 20
Right(None)
Expect:	If you survive, you will be 10'
	Right(None)

Expect: Left('Birth date could not be parsed')
Output: Left(Birth date could not be parsed)


- Lifting a function means transforming a <u>normal</u> function into a <u>higher-order</u> function that works within a functor, monad, or another computational context.
- This allows functions to be applied to values wrapped inside structures like <u>Maybe</u>, <u>Either</u> without manually unpacking them.

Next, we introduce little ```either```:

In [22]:
# id :: identity a -> a
identity = lambda x: x

# either :: (a -> c) -> (b -> c) -> Either a b -> c
either = curry(lambda f, g, e: f(e.value) if isinstance(e, Left) else g(e.value) if isinstance(e, Right) else None)

# zoltar :: User -> _
zoltar = compose(print, either(identity, fortune), getAge(moment()))

In [27]:
e1 = zoltar({ 'birthDate': '2005-12-12' })
print(f"{e1}\nExpect: 'If you survive, you will be 20'\n\tNone\n")

e2 = zoltar({ 'birthDate': 'balloons!' })
print(f"{e2}\nExpect: 'Birth date could not be parsed'\n\tNone")

del e1, e2

If you survive, you will be 20
None
Expect: 'If you survive, you will be 20'
	None

Birth date could not be parsed
None
Expect: 'Birth date could not be parsed'
	None


Identity function was finaly used.<br>
Combined with ```either```, it becomes a great tool to skip morphisms when the conditions are not met !<br><br>

##### <b>Summary:</b>
```Either``` – <b>Handles computations that can fail</b>
- Represents <b>two possible outcomes</b>: ```Right(value)``` (success) or ```Left(error)``` (failure).
- Avoids exceptions by propagating errors as values.

---

#### Old McDonald Had Effects...

